In [1]:
import keras
import numpy as np
import os
import pickle
import cv2

Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import ImageDataGenerator

In [3]:
#Training generator with augmentation

train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        vertical_flip=False,
        fill_mode='nearest')

In [4]:
# Testing generator

test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
BATCH_SIZE = 256
N_CLASSES = 3036
LR = 0.001
N_EPOCHS = 50
N_UNITS = 128
IMG_SIZE = 96

In [6]:
train_generator = train_datagen.flow_from_directory(
        'training_data',  # this is the target directory
        target_size=(IMG_SIZE, IMG_SIZE),  # all images will be resized to 150x150
        batch_size=BATCH_SIZE,
        class_mode='sparse')

Found 576840 images belonging to 3036 classes.


In [7]:
validation_generator = test_datagen.flow_from_directory(
        'testing_data',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='sparse')

Found 30360 images belonging to 3036 classes.


In [8]:
from keras.applications.mobilenet import MobileNet
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.models import Sequential, Model 

model = MobileNet(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=True, classes=N_CLASSES, weights=None)

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 48, 48, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 48, 48, 32)        128       
_________________________________________________________________
conv1_relu (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 48, 48, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 48, 48, 32)        128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 48, 48, 32)        0         
__________

In [10]:
# Training

from keras.callbacks import ModelCheckpoint

model_file = "models_4/weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"

checkpoint = ModelCheckpoint(model_file, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# model.load_weights('models_3/weights-improvement-85-0.06.hdf5')

model.fit_generator(
        train_generator,
        steps_per_epoch=576840 // BATCH_SIZE,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=30360 // BATCH_SIZE,
        callbacks=callbacks_list)

Epoch 1/50
2253/2253 [==============================] - 898s 399ms/step - loss: 5.1680 - acc: 0.1007 - val_loss: 3.6318 - val_acc: 0.2318

Epoch 00001: val_loss improved from inf to 3.63177, saving model to models_4/weights-improvement-01-3.63.hdf5
Epoch 2/50
2253/2253 [==============================] - 899s 399ms/step - loss: 0.8022 - acc: 0.7814 - val_loss: 0.4588 - val_acc: 0.8706

Epoch 00002: val_loss improved from 3.63177 to 0.45881, saving model to models_4/weights-improvement-02-0.46.hdf5
Epoch 3/50
2253/2253 [==============================] - 894s 397ms/step - loss: 0.2622 - acc: 0.9269 - val_loss: 0.2270 - val_acc: 0.9375

Epoch 00003: val_loss improved from 0.45881 to 0.22699, saving model to models_4/weights-improvement-03-0.23.hdf5
Epoch 4/50
2253/2253 [==============================] - 898s 399ms/step - loss: 0.1652 - acc: 0.9541 - val_loss: 0.2404 - val_acc: 0.9329

Epoch 00004: val_loss did not improve
Epoch 5/50
2253/2253 [==============================] - 892s 396ms/s


Epoch 00040: val_loss did not improve
Epoch 41/50
2253/2253 [==============================] - 899s 399ms/step - loss: 0.0145 - acc: 0.9961 - val_loss: 0.0396 - val_acc: 0.9912

Epoch 00041: val_loss did not improve
Epoch 42/50
2253/2253 [==============================] - 898s 399ms/step - loss: 0.0138 - acc: 0.9963 - val_loss: 0.0301 - val_acc: 0.9924

Epoch 00042: val_loss improved from 0.03261 to 0.03010, saving model to models_4/weights-improvement-42-0.03.hdf5
Epoch 43/50
2253/2253 [==============================] - 898s 398ms/step - loss: 0.0139 - acc: 0.9961 - val_loss: 0.0332 - val_acc: 0.9925

Epoch 00043: val_loss did not improve
Epoch 44/50
2253/2253 [==============================] - 897s 398ms/step - loss: 0.0135 - acc: 0.9964 - val_loss: 0.0324 - val_acc: 0.9926

Epoch 00044: val_loss did not improve
Epoch 45/50
2253/2253 [==============================] - 900s 400ms/step - loss: 0.0138 - acc: 0.9963 - val_loss: 0.0334 - val_acc: 0.9922

Epoch 00045: val_loss did not imp

In [13]:
model.load_weights('models_4/weights-improvement-42-0.03.hdf5')

In [15]:
with open("model_mobilenet.json", "w") as json_file:
    json_file.write(model.to_json())